In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import ee
import geemap
from datetime import datetime
from pathlib import Path
# import os
from osgeo import gdal

import rioxarray
import numpy as np
import glob

In [2]:
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

# Designate area of interest (AOI)

## Set up name and directory structure

This is where you can change the ID of the watershed

In [3]:
# If you're doing watersheds
HYBAS_ID = 3100083000
aoi_name = str(HYBAS_ID)

# If you're not
# aoi_name = 'test_aoi'

In [4]:
# If you want to save things locally
aoi_path = Path('.', str(aoi_name))

# If you're in the arctic group!
# aoi_path = Path('/sciclone/data10/watersheds', str(aoi_name))

Path(Path(aoi_path)).mkdir(parents=True, exist_ok=True)

## Set up area

### Example: lat/long with area buffer

In [42]:
# latitude, longitude  = (65.052164, -166.264824) # Seward Peninsula 

# latitude, longitude  =  (-77.56947545454703, 161.22678556499886) # Taylor Valley

latitude, longitude  = (68.62245827327547, -149.34257980791222) # WT6, Toolik

aoi_point = ee.Geometry.Point([longitude, latitude])

# Make your Polygon a FeatureCollection to conform to format of EE-hosted FeatureCollections
# You can still generate ImageCollections with a Polygon, but the downloading step won't work
aoi = ee.FeatureCollection(ee.Feature(ee.Geometry(aoi_point.buffer(5000).bounds())))



### Or use the HydroSHEDS geometry

In [6]:
# aoi = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_10").filter(ee.Filter.eq('HYBAS_ID', HYBAS_ID))

# longitude = aoi.geometry().centroid().coordinates().get(0).getInfo()
# latitude = aoi.geometry().centroid().coordinates().get(1).getInfo()

You can use geemap's `geemap.shp_to_ee()` function to turn a local shapefile into an AOI as well. 

# Define EE functions and get filtered ImageCollection

## Functions

In [32]:
def maskS2clouds(image):
        """Masks clouds in S2 images

    Parameters:
    image (Image): A single Image in an ImageCollection or standalone Image

    Returns:
    Image with masked features and original metadata

    """
        qa = image.select('QA60')

        # Bits 10 and 11 are clouds and cirrus, respectively.
        cloudBitMask = 1 << 10
        cirrusBitMask = 1 << 11

        # Both flags should be set to zero, indicating clear conditions.
        mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
            .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

        return image.updateMask(mask) \
            .divide(10000) \
            .copyProperties(image, ['system:time_start'])

def clp(image):
    '''Clips a single Image to a region of interest'''
    return image.clip(aoi)

def mosaicByDate(imcol):
        """Creates a mosaicked Image for a single date if there are
        multiple images from a single date 

    Parameters:
    imcol (ImageCollection): An ImageCollection with images from one or more dates

    Returns:
    ImageCollection with images mosaicked by date

        """
        # Get a list of unique dates in the image collection
        imlist = imcol.toList(imcol.size())

        unique_dates = imlist.map(lambda im: ee.Image(im).date().format("YYYY-MM-dd")).distinct().getInfo()

        # Create an empty list to store mosaic images
        mosaic_imlist = []

        # Loop through unique dates and create mosaic images
        for date_str in unique_dates:
            date = ee.Date.parse("YYYY-MM-dd", date_str)
            mosaic_image = imcol.filterDate(date, date.advance(1, "day")).mosaic()
            mosaic_image = mosaic_image.set("system:time_start", date.millis(), "system:id", date_str)
            mosaic_imlist.append(mosaic_image)

        return ee.ImageCollection(mosaic_imlist)

def addNDVI(image):
  '''Adds S2's NDVI band to each image (in an ImageCollection)'''
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

def addNDSI(image):
    '''Adds S2's NDSI band to each image (in an ImageCollection)'''
    ndsi = image.normalizedDifference(['B3', 'B11']).rename('NDSI')
    return image.addBands(ndsi)

def addElevation(image):
  '''Adds ArcticDEM elevation to each image (in an ImageCollection)'''
  elevation = ee.Image("UMN/PGC/ArcticDEM/V3/2m_mosaic").select('elevation').clip(aoi).rename('ArcticDEM')
  return image.addBands(elevation)

def get_mean_snow_cover(image):
        """Adds a value for scene-averaged MODIS-dervied snow cover to an image (in an ImageCollection)

    Parameters:
    image (Image): A single Image in an ImageCollection or standalone Image

    Returns:
    Image with snow cover mean as a band

        """    
        # Get MODIS snow cover product for day and location
        ndsi_image = ee.ImageCollection('MODIS/061/MOD10A1').filterDate(
            image.date(), image.date().advance(1, 'day')).first().select('NDSI_Snow_Cover').clip(aoi)
        
        image = image.addBands(ndsi_image)

        # Get mean value across the scene 
        mean_value = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=aoi,
            scale=100,  # Resolution of Landsat data in meters
            maxPixels = 1e9
        )

        # Get the mean value for the band
        mean_band_value = mean_value.get('NDSI_Snow_Cover')

        # Set the mean value as an image property
        return image.set("mean_" + 'NDSI_Snow_Cover', mean_band_value)

def calculateNoDataPercentage(image):
        """Add data on masked pixel percentage as a band
        note that total_pixels needs to be calculated first

    Parameters:
    image (Image): A single Image in an ImageCollection or standalone Image

    Returns:
    Image with "nodata_percentage band added"

    """
    
    # Any masked no data stuff will be equal to 1
        nodata_mask = image.select('B1').mask().eq(0)
            
        # Sum up the nodata 1's
        # nodata_pixels = nodata_mask.reduceRegion(
        #     reducer=ee.Reducer.sum(),
        #     geometry=aoi,
        #     scale=100,  # s2 res
        #     maxPixels = 1e9
        # )
        
        # # Calculate the percentage of NoData values
        # percentage_nodata = nodata_pixels.getNumber('B1').divide(total_pixels.getNumber('B1')).multiply(100)

        
        # # Set the NoData percentage as an image property
        # return image.set("nodata_percentage", percentage_nodata)  
        nodata_stats = nodata_mask.reduceRegion(
        reducer=ee.Reducer.sum().combine(
            reducer2=ee.Reducer.count(),
            sharedInputs=True
        ),
        geometry=aoi,  # Assuming 'aoi' is defined somewhere in your script
        scale=100,  # Adjust the scale according to your requirements
        maxPixels=1e9
        )

        # Extract sum and count values
        sum_nodata_pixels = nodata_stats.getNumber('B1_sum')
        count_nodata_pixels = nodata_stats.getNumber('B1_count')

        # Calculate percentage
        nodata_percentage = sum_nodata_pixels.divide(count_nodata_pixels).multiply(100)

        # Add NoData percentage as a band
        return image.set('nodata_percentage', nodata_percentage)

def get_white_pixel_percent(image):
        """Add data on pixel percentage that is white in grayscale as a band
        note that total_pixels needs to be calculated first

    Parameters:
    image (Image): A single Image in an ImageCollection or standalone Image

    Returns:
    Image with "white_percentage" band added

    """    
        grayscale = image.expression(
            '(.3 * 1e4 * R) + (.59 * 1e4 * G) + (.11 * 1e4 * B)', {
            # '(R + G + B) / 3', {
            'R': image.select('B4'),
            'G': image.select('B3'),
            'B': image.select('B2')
        })

        white_mask = grayscale.gt(2000)
        
        # white_mask needs to = 1

        # white_pixels = white_mask.reduceRegion(
        #     reducer=ee.Reducer.sum(),
        #     geometry=aoi,
        #     scale=100,  # s2 res
        #     maxPixels = 1e9
        # )

        # # # Calculate the total number of pixels within the ROI
        # # total_pixels = image.select('B1').reduceRegion(
        # #     reducer=ee.Reducer.count(),
        # #     scale=10,  # s2 res
        # #     maxPixels = 1e9
        # # )

        # # percentage_white = white_pixels.getNumber('constant').divide(total_pixels.getNumber('B1')).multiply(100)
        
        # total_pixels = image.getNumber('total_pixels')  # Get total_pixels from the image properties

        # percentage_white = white_pixels.getNumber('constant').divide(total_pixels).multiply(100)
        
        # # Set the NoData percentage as an image property
        # return image.set("white_percentage", percentage_white).set("white_pixel_count", white_pixels.getNumber('constant')) 
        white_pixel_stats = white_mask.reduceRegion(
        reducer=ee.Reducer.sum().combine(
                reducer2=ee.Reducer.count(),
                sharedInputs=True
            ),
            geometry=aoi,  # Assuming 'aoi' is defined somewhere in your script
            scale=100,  # Adjust the scale according to your requirements
            maxPixels=1e9
        )

        # Extract sum and count values
        sum_white_pixels = white_pixel_stats.getNumber('constant_sum')
        count_white_pixels = white_pixel_stats.getNumber('constant_count')

        # Calculate percentage
        white_percentage = sum_white_pixels.divide(count_white_pixels).multiply(100)

        return image.set("white_percentage", white_percentage)

def calcTotalPixels(image):
    """Add data on total pixels as a band

    Parameters:
    image (Image): A single Image in an ImageCollection or standalone Image

    Returns:
    Image with "total_pixels" band added

    """    
    total_pixels = image.select('B1').reduceRegion(
        reducer=ee.Reducer.count(),
        scale=100,  
        maxPixels = 1e9,
        geometry=aoi
    )
    return image.set("total_pixels", total_pixels.getNumber('B1')) 

def getVisibleImages(Map):
        """Retrieves names of layers visible on the Map 

    Parameters:
    Map (Map): A geemap.Map() 

    Returns:
    A list of strings corresponding to the labels on the Map layers
    if they are dates (as needed for the original notebook)

    """    
        map_layers = list(Map.layers)
        visibility_status = [layer.visible for layer in map_layers]
        visible_layers = [x.name for x, y in zip(map_layers, visibility_status) if y == True]
        return [x for x in visible_layers if '-' in x and datetime.strptime(x, '%Y-%m-%d')]


## Set filtering parameters 

If you notice that your ImageCollections are empty, try changing these!

In [33]:
## Good Arctic thresholds
snow_cover_threshold = 0
threshold_nodata_percent = 0.5
threshold_white_percent = 2

## Antarctica is much whiter
# snow_cover_threshold = 100
# threshold_nodata_percent = 50
# threshold_white_percent = 100


# Limit images added to the Map
image_limit=5

## Build ImageCollection

This step involves loading in Sentinel 2 data and applies your first filter for dates and cloudy percentage as well as filtering to images that intersect with the bounds of your area of interest and masking for clouds. 

For detecting water tracks we want to look at the growing season (months 5 to 9) but adjust based on science question. 

In [44]:
dataset = (
                ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                .filter(ee.Filter.calendarRange(2019,2023,'year'))
                .filter(ee.Filter.calendarRange(5,9,'month'))
                # Pre-filter to get less cloudy granules.
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
                # .filterBounds(aoi.centroid())'
                .filterBounds(aoi)
                .map(clp)
                .map(maskS2clouds)
)

Add any extra bands or data prior to filtering

In [45]:
dataset = mosaicByDate(dataset).map(addNDVI).map(addNDSI).map(addElevation)

collection_with_data = dataset.map(calcTotalPixels).map(calculateNoDataPercentage).map(get_white_pixel_percent)


Now filter

In [46]:
# Basically a "too many aggregated requests" error can be thrown if you do TOO much at once (like three reduceRegions and filters)
# So my strategy here is to subsume the snow filter into the white filter

filtered_collection = collection_with_data.filter(ee.Filter.lte("nodata_percentage", threshold_nodata_percent))
filtered_collection = filtered_collection.filter(ee.Filter.lte("white_percentage", threshold_white_percent))

filtered_collection_size = len(filtered_collection.aggregate_array("system:index").getInfo())

print(f'Number of images in collection: {filtered_collection_size}')

Number of images in collection: 26


In [47]:
# This line only instantiates a "Map" object from gee
# You need to make one to add layers to it
# but we don't display it yet 

# If you run this line after adding layers, 
# you will lose the layers because you made a new Map
Map = geemap.Map(center=[latitude, longitude], zoom=11)

In [48]:
for image_id in filtered_collection.aggregate_array("system:index").getInfo()[0:image_limit]:
    image = filtered_collection.filterMetadata("system:index", "equals", image_id).first()
    
    image_RGB = image.select('B4', 'B3', 'B2') 
    # image_RGB = image.select('B4') 
    RGB_vis_params = {'min': 0.0, 'max': 0.3}
    Map.addLayer(image_RGB, RGB_vis_params, ee.Image(image).date().format('yyyy-MM-dd').getInfo(), True)

print(f'Remember I am limiting the image list to {image_limit} of {filtered_collection_size} images')


Remember I am limiting the image list to 5 of 26 images


# Map

In [49]:
Map

Map(center=[68.62245827327547, -149.34257980791222], controls=(WidgetControl(options=['position', 'transparent…

Now if you click on the wrench in the right top corner, followed by the "Layers" button, you can toggle through imagery that passed the filter. 

If you notice squares missing from the imagery, it might be a rendering/tiling issue - if you zoom in and out the map will re-tile and the problem is usually solved. The data are there, I promise. 


Now you can download any imagery you want by keeping that layer visible and running the following script

In [50]:
date_layers = getVisibleImages(Map)

for date_string in date_layers:
    print(date_string)

    imageDate = ee.Date(date_string)

    to_download = filtered_collection.filterDate(imageDate).first().visualize(
        bands=['B4', 'B3', 'B2'],
        min=0.001, max=0.3
        )

    if isinstance(aoi, ee.featurecollection.FeatureCollection):
        geemap.ee_to_shp(((aoi)), filename=str(aoi_path.joinpath(f'{aoi_name}.shp')))
    
        geemap.download_ee_image(to_download, str(aoi_path.joinpath(f'{aoi_name}_{date_string}_RGB.tif')), scale=10, region=aoi.geometry(), crs='EPSG:3995')

        to_download = filtered_collection.filterDate(imageDate).first().select('NDVI')

        geemap.download_ee_image(to_download, str(aoi_path.joinpath(f'{aoi_name}_{date_string}_NDVI.tif')), scale=10, region=aoi.geometry(), crs='EPSG:3995')

        to_download = filtered_collection.filterDate(imageDate).first().select('ArcticDEM')

        geemap.download_ee_image(to_download, str(aoi_path.joinpath(f'{aoi_name}_ArcticDEM.tif')), scale=2, region=aoi.geometry(), crs='EPSG:3995')

    else:
        print("Your AOI is not a FeatureCollection, using backup routine.")

        geemap.ee_to_shp(ee.FeatureCollection(ee.Feature(ee.Geometry(aoi))), filename=str(aoi_path.joinpath(f'{aoi_name}.shp')))
  
        geemap.download_ee_image(to_download, str(aoi_path.joinpath(f'{aoi_name}_{date_string}_RGB.tif')), scale=10, region=aoi, crs='EPSG:3995')

        to_download = filtered_collection.filterDate(imageDate).first().select('NDVI')

        geemap.download_ee_image(to_download, str(aoi_path.joinpath(f'{aoi_name}_{date_string}_NDVI.tif')), scale=10, region=aoi, crs='EPSG:3995')

        to_download = filtered_collection.filterDate(imageDate).first().select('ArcticDEM')

        geemap.download_ee_image(to_download, str(aoi_path.joinpath(f'{aoi_name}_ArcticDEM.tif')), scale=2, region=aoi, crs='EPSG:3995')


2019-06-01


test_aoi_2019-06-01_RGB.tif: |          | 0.00/5.73M (raw) [  0.0%] in 00:00 (eta:     ?)

test_aoi_2019-06-01_NDVI.tif: |          | 0.00/7.64M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: 2019-06-01


test_aoi_2019-06-01_ArcticDEM.tif: |          | 0.00/191M (raw) [  0.0%] in 00:00 (eta:     ?)

## Perform additional topographic metric calculations

Between this step and the next, you could perform any number of additional analyses and save the results as a geotiff of any resolution (e.g. flow accumulation, curvature). As long as the file ends up in the directory you downloaded these images to, the next steps will work. 

# Align

Coming soon: a glob that also globs for dates (in case you put multiple days' images for the same AOI in the same directory.)

In [ ]:
aligned_path = Path(aoi_path, 'aligned')
Path(aligned_path).mkdir(parents=True, exist_ok=True)

The variable `res_flag` sets whether you want to match the lowest resolution in your dataset (`"max"`) or your highest (`"min"`). For example, ArcticDEM is 2 m but Sentinel 2 RGB is 10 m. `'min'` would make everything 2 m (which would split Sentinel 2 pixels up), and `'max'` would make everything 10 m (which would coarsen the ArcticDEM). 

In [ ]:
# Save your generator object
tif_list = list(aoi_path.glob('*.tif'))

ds_list = []

res_flag = 'max'

The following script creates a folder "`aligned`" within the main imagery directory

In [ ]:
for tif in tif_list:
    # Changing data type for consistent nan 
    # revisit
    src = rioxarray.open_rasterio(tif).astype('float32')
    src.attrs['_FillValue'] = np.nan
    ds_list.append(src)
res_list = [src.rio.transform()[0] for src in ds_list]

if res_flag == 'min':
    # This finds dataset with minimum resolution and makes a list with everything BUT that
    to_align = ds_list[:(res_list).index(min(res_list))] + ds_list[(res_list).index(min(res_list)) + 1:]
    to_name = tif_list[:(res_list).index(min(res_list))] + tif_list[(res_list).index(min(res_list)) + 1:]

    # This aligns each of the "other" datasets to the minimum-resolution dataset
    # and **writes over** the EE-downloaded data 
    for i, src in enumerate(to_align):
        aligned = src.rio.reproject_match(ds_list[(res_list).index(min(res_list))]).rio.to_raster(Path(aligned_path, to_name[i].name))

elif res_flag == 'max':
    # This finds dataset with maximum resolution and makes a list with everything BUT that
    to_align = ds_list[:(res_list).index(max(res_list))] + ds_list[(res_list).index(max(res_list)) + 1:]
    to_name = tif_list[:(res_list).index(max(res_list))] + tif_list[(res_list).index(max(res_list)) + 1:]

    # This aligns each of the "other" datasets to the minimum-resolution dataset
    # and **writes over** the EE-downloaded data 
    for i, src in enumerate(to_align):
        aligned = src.rio.reproject_match(ds_list[(res_list).index(max(res_list))]).rio.to_raster(Path(aligned_path, to_name[i].name))

# Add the "template" geotiff to the directory for making jpegs
source = [source_name for source_name in set(tif_list).difference(to_name)][0]

source_path = Path(source)
destination_path = Path(aligned_path) / source_path.name
# source_path.replace(destination_path)

In [ ]:
## PREVIEW: export as xarray dataset
## and then play with it further

# import xarray as xr

# # Create a new xarray dataset with aligned data
# ds = xr.Dataset({
#     'dem': dem,
#     'red': rgb.sel(band=1),
#     'green': rgb.sel(band=2),
#     'blue': rgb.sel(band=3),
#     'ndvi': ndvi
# })
# ds['ndvi'].rio.to_raster('output/aligned_ndvi_2.tif')

# Tile

The following script will make a directory "`uint8`" within the "`aligned`" directory because you need to scale every aligned geotiff to 0-255 to convert it to jpeg.

Note there is probably a world in which something like `tensorflow` does not need jpegs but instead just `.npz` files...a work in progress! But for now this plugs seamlessly into `segmentation_gym`. 

In [ ]:
input_dir = Path(aligned_path)
unit_dir = Path('./test_aoi/aligned/uint8/')

# Create the output directory if it doesn't exist
unit_dir.mkdir(parents=True, exist_ok=True)

# Get a list of all GeoTIFF files in the input directory
geotiff_files = list(input_dir.glob('*.tif'))

for geotiff_file in geotiff_files:
    input_path = str(geotiff_file)
    print(f'Reading {input_path}')
    # Open the input GeoTIFF file
    src_ds = gdal.Open(input_path)
    if src_ds is None:
        print(f"Failed to open {input_path}")
        continue

    # Read the data as Float32
    float32_data = src_ds.ReadAsArray()

    # Scale and convert the data to UInt8
    uint8_data = (float32_data - np.nanmin(float32_data)) / (np.nanmax(float32_data) - np.nanmin(float32_data))
    uint8_data = (uint8_data * 255).astype(np.uint8)

    # Create a new GeoTIFF file with UInt8 data
    output_path = unit_dir / geotiff_file.name
    ds = gdal.GetDriverByName('GTiff').Create(
        str(output_path),
        src_ds.RasterXSize,
        src_ds.RasterYSize,
        src_ds.RasterCount,  # Number of bands
        gdal.GDT_Byte  # Data type: UInt8
    )

    # Write the UInt8 data to the bands
    # For single band
    if src_ds.RasterCount == 1:
        ds.GetRasterBand(1).WriteArray(uint8_data)
    else:
        # For multiband RGB
        for band_num in range(1, src_ds.RasterCount + 1):
            ds.GetRasterBand(band_num).WriteArray(uint8_data[band_num - 1])


    # Set the original spatial reference
    ds.SetProjection(src_ds.GetProjection())
    ds.SetGeoTransform(src_ds.GetGeoTransform())

    ds.FlushCache()

    # Close the datasets
    ds = None
    src_ds = None

And finally a directory is created called "`jpegs`" inside "`uint8`". 

In [ ]:
def split_into_jpeg_tiles(input_path, output_dir, tile_size):
    """Takes a geotiff and splits it into jpeg tiles perfect for loading into
    Doodleverse tools or any image segmentation dataset/training scheme. In the process of creating
    (non-georeferenced) jpeg tiles it will export a .xml file for each tile so that the jpegs,
    or any labels created with Doodler etc. can be re-georeferenced if desired. 

    Parameters:
    input_path: path for geotiff you want to tile up
    output_dir: the directory you want to stick your jpeg tiles
    tile_size: N x N pixels per tile

    Returns:
    Nothing, but writes files to the output_dir

    """

    fname = input_path.stem

    output_format = 'JPEG'
    creation_options = ['QUALITY=95']

    input_path = Path(input_path)
    output_dir = Path(output_dir)

    src_ds = gdal.Open(str(input_path))

    width = src_ds.RasterXSize
    height = src_ds.RasterYSize
    print(f"{fname} is {width} x {height}")

    cols = width // tile_size
    rows = height // tile_size

    for row in range(rows):
            for col in range(cols):
                x_offset = col * tile_size
                y_offset = row * tile_size

                # Adjust the width and height for the last row and column
                tile_width = min(tile_size, width - x_offset)
                tile_height = min(tile_size, height - y_offset)

                output_file = output_dir / f"{fname}_{row}_{col}.jpeg"

                ds = gdal.Translate(
                    str(output_file),
                    str(input_path),
                    srcWin=[x_offset, y_offset, tile_width, tile_height],
                    format=output_format,
                    width=tile_width,
                    height=tile_height,
                    creationOptions=creation_options
                )

                # Close the output dataset
                ds = None

    # Close the source dataset
    src_ds = None

In [ ]:
tif_list = list(unit_dir.glob('*.tif'))
tile_size = 256
output_directory = './test_aoi/aligned/uint8/jpegs/'
output_directory = Path(output_directory)
output_directory.mkdir(parents=True, exist_ok=True)

for tif in tif_list:
    input_geotiff = tif

    split_into_jpeg_tiles(input_geotiff, output_directory, tile_size)

So the structure is:
```
some/place/you/are/working
                    |   |   ├── your_aoi_string
                    |   │   |       └── *.tif
                    |   |   |     ├──aligned
                    |   │   |       └── *.tif
                    |   |   |   |     ├──uint8
                    |   |   │   |       └── *.tif
                    |   |   |   |   |     ├──jpegs
                    |   |   |   │   |       └── *N_N.jpeg
                    |   |   |   │   |       └── *N_N.jpeg.aux.xml
          

```

where `N` is a row or column number of the original geotiff

Now you can copy the RGB jpegs into your "`assets`" folder for Doodler and save the other outputs as other data layers for Segmentation Gym. Future iterations will automatize this part, as well as show an example of bringing in your own labels from GIS or other shapefiles if you want to skip Doodling. 